In [47]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [48]:
month="jul2023"

In [49]:
def loop(file_name):
    return pd.read_csv(f"{month}/urban/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')

In [50]:
def NonGOVT(df):
    govt = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,72,73,74,75,80,81,82,101,102,103,104,105,106,107,'80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [51]:
def billable(df):
    bil = ['In Service','TD','TD >6 MONTHS']
    df['bilable_flag'] = df['CON_STATUS'].isin(bil)
    return df
  

In [52]:
def arrear(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['ARREAR_FILTER'] = None
    df.loc[df['TOTAL_OUTSTANDING'] < 5000,"ARREAR_FILTER"] = "<5k"
    df.loc[df['TOTAL_OUTSTANDING'] >= 5000,"ARREAR_FILTER"] = ">=5k"
#     df.loc[((df['TOTAL_OUTSTANDING'] > 10000) & (df['TOTAL_OUTSTANDING'] <= 100000)),"ARREAR_FILTER"] = "10k-1L"
#     df.loc[((df['TOTAL_OUTSTANDING'] > 100000) & (df['TOTAL_OUTSTANDING'] <= 5000000)),"ARREAR_FILTER"] = "1L-50L"
#     df.loc[df['TOTAL_OUTSTANDING'] > 5000000,"ARREAR_FILTER"] = "GRET_50L"
    
    return df

In [53]:
def CatLoad(df):
    df['LOAD'] = pd.to_numeric(df['LOAD'],errors="coerce")
    df['LOAD_FILTER'] = None
    df.loc[df['LOAD'] < 5,"LOAD_FILTER"] = "0-5kw"
    df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['LOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df

    

In [54]:
def categories(df):
    #Supplytype.srs.startswith('1') for lmv1
    df['Cat'] = None
#     df[df['SUPPLY_TYPE'].str.startswith('1')] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('1', na=False),"Cat"] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('2',na=False),"Cat"] = "LMV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('3', na=False),"Cat"] = "LMV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('4', na=False),"Cat"] = "LMV4"
    df.loc[df['SUPPLY_TYPE'].str.startswith('5', na=False),"Cat"] = "LMV5"
    df.loc[df['SUPPLY_TYPE'].str.startswith('6', na=False),"Cat"] = "LMV6"
    df.loc[df['SUPPLY_TYPE'].str.startswith('7', na=False),"Cat"] = "LMV7"
    df.loc[df['SUPPLY_TYPE'].str.startswith('8', na=False),"Cat"] = "LMV8"
    df.loc[df['SUPPLY_TYPE'].str.startswith('9', na=False),"Cat"] = "LMV9"
#     df.loc[df['SUPPLY_TYPE'].str.startswith('10'),"Cat"] = "LMV10"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H1', na=False),"Cat"] = "HV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H2', na=False),"Cat"] = "HV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H3', na=False),"Cat"] = "HV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H4', na=False),"Cat"] = "HV4"
    return df

In [55]:
def date(df):
    df['LAST_PAY_DATE'] = pd.to_datetime(df['LAST_PAY_DATE'],format='%d-%b-%Y',errors = "coerce")
    df['DOC'] = pd.to_datetime(df['DOC'],format='%d-%b-%Y',errors = "coerce")
    df['LAST_BILL_DATE'] = pd.to_datetime(df['LAST_BILL_DATE'],format='%d-%b-%Y',errors = "coerce")
    df['LAST_OK_READ_STATUS'] = pd.to_datetime(df['LAST_OK_READ_STATUS'],format='%d-%b-%Y',errors = "coerce")
    
    return df

In [56]:
def unpaid(df):
    df['unpaid_status'] = None
    df.loc[df['LAST_PAY_DATE'] < '01-01-2023',"unpaid_status"] = "unpaid"
    df.loc[pd.isnull(df['LAST_PAY_DATE']),"unpaid_status"] = "neverpaid"     #df[~pd.isnull(df['Courses'])]
    df.loc[df['LAST_PAY_DATE'] >= '01-01-2023',"unpaid_status"] = "paid"
    
    return df

In [57]:
import os

mylist = os.listdir(f"{month}/urban")

In [58]:
mylist

['MASTER_MVVNL_DIV354921_JUL_2023.csv',
 'MASTER_MVVNL_DIV354922_JUL_2023.csv',
 'MASTER_MVVNL_DIV355131_JUL_2023.csv',
 'MASTER_MVVNL_DIV355211_JUL_2023.csv']

In [59]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
df = pd.concat(df, ignore_index=True)

end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
C:\Users\HP\AppData\Local\Temp\ipykernel_16828\1044843435.py:2: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.9s
C:\Users\HP\AppData\Local\Temp\ipykernel_16828\1044843435.py:2: DtypeWarning: Columns (3,10,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}",encoding='ISO-8859-1',on_bad_lines='skip')
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    1.5s remaining:    1.5s
C:\Users\HP\AppData\Local\Temp\ipykernel_16828\1044843435.py:2: DtypeWarning: Columns (3,10,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}",encoding='ISO-8859-1',on_bad_lines

CSV: 3.549250841140747


In [60]:
column =list(df.columns)

In [61]:
df.CON_STATUS.unique()

array(['In Service', 'TD', 'Null', 'PD', 'NC Reject', 'TD Migrated',
       'COT', 'TD >6 MONTHS', 'PD Initiated', 'NC Pending',
       'COT Initiated', 'REJECTED', 'SB'], dtype=object)

In [62]:
column

['DIV_CODE',
 'SDO_CODE',
 'ACCT_ID',
 'KNO',
 'MOBILE_NO',
 'LANDLINE_NO',
 'BOOK_NO',
 'SCNO',
 'NAME',
 'ADDRESS',
 'SUPPLY_TYPE',
 'LOAD',
 'LOAD_UNIT',
 'DOC',
 'SECURITY_AMT',
 'CON_STATUS',
 'SERIAL_NBR',
 'MULTIPLY_FACTOR',
 'METER_STATUS',
 'LAST_BILL_DATE',
 'CLOSE_READING',
 'MDI',
 'BILL_BASIS',
 'BILL_TYP',
 'CONSUMPTION_CURR_MNTH',
 'CONSUMPTION_PREV_MNTH',
 'CONSUMPTION_PREV_TO_PREV_MNTH',
 'ARREAR',
 'LPSC',
 'CURRENT_ASSESSMENT',
 'CURRENT_CYCLE_LPSC',
 'TOTAL_OUTSTANDING',
 'DUE_DATE_REBATE',
 'LAST_OK_READING',
 'LAST_OK_READ_STATUS',
 'METER_READ_FLTY_CNT',
 'LAST_PAY_AMT',
 'LAST_PAY_DATE',
 'SUBSTATION',
 'FEEDER',
 'DT',
 'POLE_NO',
 'OPR_FLG',
 'BILL_AFTER_DATE',
 'METER_READ_REMARK',
 'INSTALLATION_DATE',
 'SBM_BILL_DATE',
 'SBM_MACHINE_ID',
 'BILL_CYC_CD',
 'TOWN',
 'SERVICE_CYC_CD',
 'CT_RATIO',
 'PT_RATIO',
 'EC',
 'FC',
 'REBATES',
 'MIN_CHARGES',
 'FUEL_SURCHARGE',
 'DEMAND_AMT',
 'LTMETERING_CHARGES',
 'CAP_CHARGES',
 'REG_SURCHARGE',
 'REG_SURCHARGE_2',


In [63]:
im_col = ['DIV_CODE',
          'SDO_CODE',
          'ACCT_ID',
          'SUBSTATION',
          'FEEDER',
    
          'NAME',
          'ADDRESS',
          'MOBILE_NO',
          'SUPPLY_TYPE',
          'CONSUMPTION_CURR_MNTH',
          
          'DOC',
          'BILL_BASIS',
          'METER_STATUS',
          'METER_READ_REMARK',
          'CON_STATUS',
          'LOAD',
          'MDI',
          'ARREAR',
          'CURRENT_ASSESSMENT',
          'TOTAL_OUTSTANDING',
          
          'TOTAL_PAY_AMT']

In [64]:
df = df[im_col]

In [65]:
df = NonGOVT(df)
# df = date(df)
# df = unpaid(df)
df = billable(df)
# df['TOTAL_OUTSTANDING'] = pd.to_numeric(df['TOTAL_OUTSTANDING'],errors="coerce")
# df = arrear(df)
df = categories(df)
# df=CatLoad(df)
df['month'] = month

In [66]:
df['LOAD'] = pd.to_numeric(df['LOAD'],errors="coerce")
df['MDI'] = pd.to_numeric(df['MDI'],errors="coerce")

In [67]:
df1 = df[df['MDI'] > df['LOAD']]

In [68]:
df1.to_csv(f"load/urban/{month}.csv")

TypeError: '>' not supported between instances of 'str' and 'int'

<ipython-input-22-419b918d01a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ARREAR_FILTER'] = None
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,DIVISION,SCNO,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,CAT_CODE,ADDRESS1,ADDRESS2,ADDRESS3,...,GOVT_FLAG,GOVT_CODE,GOVT_TYPE,PAYMENT_TYPE,AMOUNT_PAID_IN_BILLMONTH,SCHEME,OPR_FLAG,gov_flag,bilable_flag,ARREAR_FILTER
0,EDD-II-BAHRAICH-NANPARA,NP10403055841,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,BABAGANJ,17,LMV-1,SORAHIYA,BABAGANJ,NNP,...,GOVERNMENT,G,CG,NaN,0,NaN,Y,False,True,10k-1L
3,EDD-II-BAHRAICH-NANPARA,NP10402045026,EDSD-I BAHRAICH NANPARA,SAHABA/BABAGANJ,RUPAIDEEHA,17,LMV-1,BAHRAICH,HASMAT ALI,RUPAIDEHA,...,GOVERNMENT,G,CG,NaN,0,NaN,Y,False,True,0-10k
4,EDD-II-BAHRAICH-NANPARA,NP10103305121,EDSD-I BAHRAICH NANPARA,NANPARA,MIHIPURWA,17,LMV-1,VITAHNIYA,NANPARA,NaN,...,Central Government,CG,CG,NaN,0,NaN,Y,False,True,0-10k
5,EDD-II-BAHRAICH-NANPARA,NP10417203506,EDSD-I BAHRAICH NANPARA,NaN,NaN,17,LMV-1,KALLYANPUR,TRILOKPUR,NaN,...,Central Government,CG,CG,NaN,0,Saubhagya,Y,False,True,0-10k
6,EDD-II-BAHRAICH-NANPARA,NP10417203711,EDSD-I BAHRAICH NANPARA,NaN,NaN,17,LMV-1,KALLYANPUR,TRILOKPUR,NaN,...,Central Government,CG,CG,NaN,0,Saubhagya,Y,False,True,10k-1L


,PHONE,CONTRACTLOAD,TOTBILL,CURTOTAMT,TOTPAYAMT,AMOUNT,ACCOUNTNO,LAST_PAID_AMT,AMOUNT_PAID_IN_BILLMONTH
count,3.431170e+05,370379.000000,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05,3.703790e+05
mean,8.995727e+09,1.101048,6.286466e+02,6.286466e+02,1.387477e+04,1.386039e+04,7.388239e+11,1.690594e+03,2.686382e+02
std,6.617287e+09,10.147789,2.787247e+04,2.787247e+04,4.976595e+04,4.953997e+04,1.258189e+10,1.394016e+04,2.803506e+04
min,0.000000e+00,0.040000,-8.910830e+05,-8.910830e+05,-5.613442e+06,-5.613442e+06,7.316000e+11,0.000000e+00,0.000000e+00
25%,8.291664e+09,1.000000,1.294500e+02,1.294500e+02,2.429530e+03,2.425000e+03,7.318135e+11,0.000000e+00,0.000000e+00
50%,9.450258e+09,1.000000,3.188400e+02,3.188400e+02,7.307610e+03,7.306000e+03,7.319047e+11,1.000000e+02,0.000000e+00
75%,9.836537e+09,1.000000,6.343300e+02,6.343300e+02,1.185939e+04,1.185600e+04,7.322001e+11,1.526000e+03,0.000000e+00
max,1.000000e+12,4450.000000,1.513922e+07,1.513922e+07,1.512410e+07,1.513922e+07,7.617023e+11,5.164381e+06,1.498798e+07


In [33]:
df.dtypes

DIV_CODE                         object
SDO_CODE                         object
ACCT_ID                           int64
SUBSTATION                       object
FEEDER                           object
NAME                             object
ADDRESS                          object
MOBILE_NO                       float64
SUPPLY_TYPE                      object
CONSUMPTION_CURR_MNTH           float64
CONSUMPTION_PREV_MNTH           float64
DOC                      datetime64[ns]
BILL_BASIS                       object
METER_STATUS                     object
METER_READ_REMARK                object
CON_STATUS                       object
LOAD                            float64
ARREAR                          float64
CURRENT_ASSESSMENT              float64
TOTAL_OUTSTANDING               float64
LAST_OK_READING                 float64
LAST_OK_READ_STATUS      datetime64[ns]
METER_READ_FLTY_CNT             float64
LAST_PAY_AMT                    float64
LAST_PAY_DATE            datetime64[ns]


0         30-06-2022
1         14-07-2022
2         21-06-2022
3         11-08-2022
4         30-08-2022
             ...    
425441    28-08-2022
425443    16-03-2020
425444    21-03-2021
425445           NaN
425446    19-12-2018
Name: LAST_PAID_DATE, Length: 370379, dtype: object

In [20]:
# df['LAST_PAID_DATE'] = pd.to_datetime(df.LAST_PAID_DATE)
# df['SUPP_RELEASE_DATE'] = pd.to_datetime(df.SUPP_RELEASE_DATE)

DIVISION                     object
SUBDIVISION                  object
SUBSTATION                   object
FEEDER                       object
SUPPLY_TYPE                  object
ADDRESS1                     object
ADDRESS2                     object
ADDRESS3                     object
NAME                         object
PHONE                       float64
BILLBASIS                    object
CONTRACTLOAD                float64
TOTBILL                     float64
CURTOTAMT                   float64
TOTPAYAMT                   float64
AMOUNT                      float64
BILLING_STATUS               object
SUPP_RELEASE_DATE            object
BILLED_CATEGORY              object
ACCOUNTNO                     int64
MTR_STATUS                   object
LAST_PAID_AMT               float64
LAST_PAID_DATE               object
GOVT_FLAG                    object
GOVT_CODE                    object
GOVT_TYPE                    object
PAYMENT_TYPE                 object
AMOUNT_PAID_IN_BILLMONTH    

array(['02-06-2022', '28-01-2021', '12-09-2022', ..., nan, nan,
       '15-02-2020'], dtype=object)

In [65]:
df1 = df[df["LOAD_FILTER"] == "great than 10kw"]

In [35]:
# inner_join = pd.merge(left=df,right=df1,on ='ACCOUNTNO',how='right')
df1

,DIV_CODE,SDO_CODE,ACCT_ID,SUBSTATION,FEEDER,NAME,ADDRESS,MOBILE_NO,SUPPLY_TYPE,CONSUMPTION_CURR_MNTH,...,ACCT_INFLALTED_FLAG,GOVT_CODE,LAST_BILL_DATE,TOTAL_PAY_AMT,gov_flag,unpaid_status,bilable_flag,ARREAR_FILTER,Cat,LOAD_FILTER
1,DIV354921,SDO3549212,5616565304,DEISEL POWER HOUSE_30165,HOSPITAL_35492120165HOS,M/S RELAINCE JIYO INFOCOM LTD,GHASYARIPURA BAHRAICH BAHRAICH Bahraich UP-271...,7.704904e+09,22,1493.0,...,N,NaN,NaT,17752.00,False,neverpaid,True,<5k,LMV2,great than 10kw
78,DIV354921,SDO3549212,2811504000,BAXIPURA_34118,TOWN-5_35492124118TO5,ATUL MATANHELIYA,VILL AMEENPVR NAGRAUR BAHRAICH UP IND,9.415054e+09,64,767.0,...,N,NaN,NaT,9600.00,False,neverpaid,True,>=5k,LMV6,great than 10kw
86,DIV354921,SDO3549212,4903941847,NaN,NaN,M/S LALIT OM FOOD PRODUCT PVT LTD PROP SRI VIN...,KATRA BHADURGANJ BHINGA ROAD BAHRAICH Bahraich...,9.415037e+09,H21T,206639.0,...,N,NaN,NaT,1643227.00,False,neverpaid,True,<5k,HV2,great than 10kw
102,DIV354921,SDO3549212,5223604000,BASHIRGANJ_30166,KILLA FEEDER_35492120166KIL,EXENG WATER,JAL NIGAM KILA BAHRICHA BAHRAICH UP IND,1.000000e+10,74,4812.0,...,Y,4101,NaT,NaN,True,neverpaid,True,>=5k,LMV7,great than 10kw
289,DIV354921,SDO3549212,1671704000,HOSPITAL CHAURAHA_34114,SANGRAHAN_35492124114SGN,MAHESH AGARWAL,GHASIYARIPURA BAHRAICHBAHRAICH BAHRAICH UP IND,9.415411e+09,11,417.0,...,N,NaN,NaT,3168.00,False,neverpaid,True,<5k,LMV1,great than 10kw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152791,DIV355211,SDO3552111,1157684000,JHANJARI_34089,BARGAON-II_35521114089BR2,DRPUNYODYA MISRA,302 PATEL NAGAR GONDAGONDA GONDA UP IND,9.415121e+09,22,6830.0,...,N,NaN,2023-02-05,70596.00,False,paid,True,<5k,LMV2,great than 10kw
152840,DIV355211,SDO3552111,148961407,AMBEDKAR CHAURAHA_34101,VISHNUPURI_35521114101VNI,SHRIRAM TRANSPORT FINANCE COMPANY LTD CO ANSHU...,34 CIVIL LINE LUCKNOW RAOD CIVIL LINE GONDA UP...,7.905403e+09,22,723.0,...,N,NaN,2023-02-03,8802.00,False,paid,True,<5k,LMV2,great than 10kw
152844,DIV355211,SDO3552111,441662768,JAIL ROAD GONDA_30170,TOWN II_35521110170TO2,M/S KAPOOR INFRASTRUCTURES C/O AKSHAT KAPOOR,1039 CIVIL LINE JAIL ROAD GONDA 1039 CIVIL LIN...,9.936102e+09,H11,19296.0,...,N,NaN,2023-02-14,NaN,False,neverpaid,True,>=5k,HV1,great than 10kw
152944,DIV355211,SDO3552111,29084277,BADAGAON_34103,DISTRICT HOSPITAL_35521114103DTL,MALHOTRA DETERGENT AND CHEMICALS OPC PVT LTD P...,300 CHHEDIPURWA STATION ROAD NEAR GGIC SCHOOL ...,9.532337e+09,22,0.0,...,N,NaN,2022-06-09,NaN,False,neverpaid,True,>=5k,LMV2,great than 10kw


In [134]:
# inner_join

,DIVISION,BILLBASIS,SUBDIVISION,SUBSTATION,KYC_CREATE_DATE,DISCONNECTION_DT,DISCONNECTION_CREATE_DT,ACCOUNTNO,Sub-div,DISCONNECTION DONE_BY
0,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,12-09-22,NaN,NaN,731911815545,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
1,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,NaN,NaN,NaN,731904554728,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
2,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,TEJWAPUR_354923,NaN,NaN,NaN,731902044463,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
3,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,TEJWAPUR_354923,NaN,NaN,NaN,732001684168,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
4,EDD-III-BAHRAICH-KAISERGANJ,MU,EDSD-III-BAHRAICH-FAKHARPUR,FAKHARPUR_354923,NaN,NaN,NaN,731812748720,EDSD-III-BAHRAICH-,JE-DIST-SUNIL \tKUMAR
...,...,...,...,...,...,...,...,...,...,...
6357,EDD-TULSIPUR,NaN,EDSD-UTRAULA,UTRAULA TAHSIL_355133,NaN,NaN,NaN,732000629570,EDSD-UTRAULA,JE-DIST-Vijay Ranjan
6358,EDD-TULSIPUR,NaN,EDSD-REHRA,ACHALPUR CHAUDHARY_355133,NaN,NaN,NaN,731703564326,EDSD-REHRA,JE-DIST-Brijnandan\tyadav
6359,EDD-TULSIPUR,NaN,EDSD-REHRA,SUB-STATION REHRA_355133,NaN,NaN,NaN,731818412228,EDSD-REHRA,SDO-DIST-Raj Kumar Yadav
6360,EDD-TULSIPUR,NaN,EDSD-REHRA,SUB-STATION REHRA_355133,NaN,NaN,NaN,731901388754,EDSD-REHRA,JE-DIST-ANAY\tKUMAR


In [44]:
start = time.time()
# df.to_csv("EDC_new.CSV")
df.to_excel("aprail2022/zone/rap_gonda.xlsx") 
# inner_join.to_excel("dec/zone/NON_GONDA_disc_jan.xlsx")

end = time.time()
print("CSV:", end - start)

CSV: 39.80852723121643


In [24]:

# # import pandas
# import pandas as pd
   
# # read csv data
# df1 = pd.read_csv('Student_data.csv')
# df2 = pd.read_csv('Course_enrolled.csv')
   
# Left_join = pd.merge(df1, 
#                      df2, 
#                      on ='Name', 
#                      how ='left')
# Left_join

FileNotFoundError: [Errno 2] File Student_data.csv does not exist: 'Student_data.csv'